# London Fire Incidents Analysis

In [1]:
%matplotlib inline 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the data
import data_download

# data_dir = "LFB-data"
LFB_data = pd.read_csv("LFB Incident data - Datastore - with notional cost and UPRN from January 2009.csv")
LFB_data.head()

Starting download ...


100%|██████████| 104M/104M [31:29<00:00, 57.6kB/s]   




patool: Extracting archive.zip ...
patool: ... archive.zip extracted to `archive' (local file exists).

Extract complete


In [ ]:
LFB_data.shape

In [ ]:
LFB_data.info()

###### Lets Clean the data.

In [ ]:
# Lets see if the data has missing values.
cols = LFB_data.columns[:]
colours = ['#000099', '#ffff00'] # specify the colours - yellow is missing. blue is not missing.
sns.heatmap(LFB_data[cols].isnull(), cmap=sns.color_palette(colours))
print("Yellow - Missing Values\nBlue - Non Missing")

In [ ]:
# Dividing the data into numerical and categorical for easier cleaning
# select numeric columns
df_numeric = LFB_data.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)


In [ ]:
df_numeric

In [ ]:

# select non numeric columns
df_categorical = LFB_data.select_dtypes(exclude=[np.number])
non_numeric_cols = df_categorical.columns.values
print(non_numeric_cols)

In [ ]:
df_categorical.isnull().sum()

We can see that we have a couple of categorical columns with missing values

In [ ]:
df_categorical.head()

In [ ]:
df_categorical.value_counts()